In [1]:
from google import genai
from pydantic import BaseModel

In [2]:
# Propositional Logic Operators

def NOT(p):
    return not p

def AND(p, q):
    return p and q

def OR(p, q):
    return p or q

def IMPLIES(p, q):
    return (not p) or q

def IFF(p, q):
    return p == q

In [3]:
# Device and Environment States
WEATHER_FREEZING = "freezing"
WEATHER_COLD = "cold"
WEATHER_WARM = "warm"
WEATHER_HOT = "hot"

DEVICE_ON = "On"
DEVICE_OFF = "Off"
DEVICE_DIMMED = "Dimmed"
DEVICE_HEATING = "Heating"
DEVICE_COOLING = "Cooling"
DEVICE_VENTILATION = "Ventilation"
DEVICE_ARMED = "Armed"
DEVICE_HOME_MODE = "Home Mode"
DEVICE_DISARMED = "Disarmed"
DEVICE_OPEN = "Open"
DEVICE_CLOSED = "Closed"

# Global knowledge base
facts = set()
rules = []

# Simplified Fact Strings
FACT_FREEZING_WEATHER = "freezing_weather"
FACT_COLD_WEATHER = "cold_weather"
FACT_WARM_WEATHER = "warm_weather"
FACT_HOT_WEATHER = "hot_weather"
FACT_USER_AT_HOME = "user_at_home"
FACT_USER_NOT_HOME = "user_not_home"
FACT_USER_LEAVING = "user_leaving"
FACT_USER_ARRIVING = "user_arriving"
FACT_IS_TIME_7 = "is_time_7"
FACT_IS_DAYTIME = "is_daytime"
FACT_USER_WANTS_LAMP_ON = "user_wants_lamp_on"
FACT_USER_WANTS_LAMP_OFF = "user_wants_lamp_off"
FACT_USER_WANTS_LAMP_DIMMED = "user_wants_lamp_dimmed"
FACT_USER_WANTS_HEATING = "user_wants_heating"
FACT_USER_WANTS_COOLING = "user_wants_cooling"
FACT_USER_WANTS_VENTILATION = "user_wants_ventilation"
FACT_USER_WANTS_THERMOSTAT_OFF = "user_wants_thermostat_off"
FACT_USER_WANTS_SECURITY_ARMED = "user_wants_security_armed"
FACT_USER_WANTS_SECURITY_HOME_MODE = "user_wants_security_home_mode"
FACT_USER_WANTS_SECURITY_DISARMED = "user_wants_security_disarmed"
FACT_USER_WANTS_TV_ON = "user_wants_tv_on"
FACT_USER_WANTS_TV_OFF = "user_wants_tv_off"
FACT_USER_WANTS_CURTAIN_OPEN = "user_wants_curtain_open"
FACT_USER_WANTS_CURTAIN_CLOSED = "user_wants_curtain_closed"

In [4]:
# System state management
state = {
    "weather": WEATHER_WARM,
    "user_at_home": True,
    "is_time_7": False,
    "is_daytime": True,
    "lamp": DEVICE_OFF,
    "thermostat": DEVICE_OFF,
    "smart_security": DEVICE_DISARMED,
    "television": DEVICE_OFF,
    "curtains": DEVICE_CLOSED,
}

def tell_fact(fact):
    facts.add(fact)
    print(f"[FACT ADDED] {fact}")

def retract_fact(fact):
    if fact in facts:
        facts.discard(fact)
        print(f"[FACT REMOVED] {fact}")
    else:
        facts.discard(fact)

def tell_rule(rule_str):
    rules.append(rule_str)
    print(f"[RULE ADDED] {rule_str}")

In [5]:
def has_fact(fact):
    return fact in facts

def print_facts():
    if facts:
        print(f"\nCurrent Facts: {sorted(list(facts))}")
    else:
        print(f"\nCurrent Facts: None")

def clear_user_request_facts():
    facts_to_remove = [fact for fact in facts if fact.startswith("user_wants_")]
    for fact in facts_to_remove:
        retract_fact(fact)

def print_state(title="STATE", show_facts=False):
    print(f"\n--- {title} ---")
    for key, value in state.items():
        print(f"{key}: {value}")
    if show_facts:
        print_facts()
    print("-" * (len(title) + 8))

In [6]:
def apply_action(action):
    action_map = {
        "turn_on_lamp": ("lamp", DEVICE_ON),
        "turn_off_lamp": ("lamp", DEVICE_OFF),
        "dim_lamp": ("lamp", DEVICE_DIMMED),
        "turn_on_heater": ("thermostat", DEVICE_HEATING),
        "turn_on_cooler": ("thermostat", DEVICE_COOLING),
        "turn_on_ventilation": ("thermostat", DEVICE_VENTILATION),
        "turn_off_thermostat": ("thermostat", DEVICE_OFF),
        "arm_security": ("smart_security", DEVICE_ARMED),
        "home_mode_security": ("smart_security", DEVICE_HOME_MODE),
        "disarm_security": ("smart_security", DEVICE_DISARMED),
        "turn_on_tv": ("television", DEVICE_ON),
        "turn_off_tv": ("television", DEVICE_OFF),
        "curtain_open": ("curtains", DEVICE_OPEN),
        "curtain_closed": ("curtains", DEVICE_CLOSED),
    }
    if action in action_map:
        device, new_state = action_map[action]
        old_state = state[device]
        state[device] = new_state
        print(f"[ACTION APPLIED] {action} -> {device}: {old_state} → {new_state}")
    else:
        print(f"[ACTION IGNORED] Unknown action: {action}")

def evaluate_premise(premise):
    premise = premise.strip()
    print(f"[PREMISE EVAL] Evaluating: '{premise}'")
    if "AND" in premise:
        parts = premise.split("AND")
        results = [evaluate_premise(part.strip()) for part in parts]
        final_result = all(results)
        print(f"[LOGICAL AND] Results: {results} -> Final: {final_result}")
        return final_result
    elif "OR" in premise:
        parts = premise.split("OR")
        results = [evaluate_premise(part.strip()) for part in parts]
        final_result = any(results)
        print(f"[LOGICAL OR] Results: {results} -> Final: {final_result}")
        return final_result
    elif premise.startswith("NOT "):
        fact = premise[4:].strip()
        fact_exists = fact in facts
        result = NOT(fact_exists)
        print(f"[LOGICAL NOT] '{fact}' exists: {fact_exists} -> NOT result: {result}")
        return result
    else:
        result = premise in facts
        print(f"[FACT CHECK] '{premise}' in facts: {result}")
        return result

def evaluate_rules():
    print(f"\n[RULE EVALUATION] Starting rule evaluation...")
    print(f"[RULE EVALUATION] Current facts: {sorted(list(facts))}")
    reset_devices_to_default()
    for rule in rules:
        print(f"\n[RULE CHECK] Evaluating rule: '{rule}'")
        if "->" in rule:
            premise, conclusion = rule.split("->", 1)
            premise = premise.strip()
            conclusion = conclusion.strip()
            print(f"[RULE CHECK] Premise: '{premise}' -> Conclusion: '{conclusion}'")
            if evaluate_premise(premise):
                print(f"[RULE FIRED] Rule triggered: {premise} -> {conclusion}")
                apply_action(conclusion)
            else:
                print(f"[RULE SKIPPED] Premise false, rule not triggered")
        elif "<->" in rule:
            left, right = rule.split("<->", 1)
            left = left.strip()
            right = right.strip()
            print(f"[BICONDITIONAL] Left: '{left}' <-> Right: '{right}'")
            left_result = evaluate_premise(left)
            right_result = evaluate_premise(right)
            if left_result == right_result:
                print(f"[BICONDITIONAL FIRED] Both sides equal ({left_result}), applying: {right}")
                apply_action(right)
            else:
                print(f"[BICONDITIONAL SKIPPED] Sides differ: {left_result} ≠ {right_result}")
    print(f"[RULE EVALUATION] Rule evaluation completed\n")

def reset_devices_to_default():
    print(f"[RESET] Resetting devices to default states")
    state["lamp"] = DEVICE_OFF
    state["thermostat"] = DEVICE_OFF
    state["smart_security"] = DEVICE_DISARMED
    state["television"] = DEVICE_OFF
    state["curtains"] = DEVICE_CLOSED

In [7]:
def update_time_facts(is_time_7, is_daytime):
    print(f"[TIME UPDATE] Setting is_time_7={is_time_7}, is_daytime={is_daytime}")
    if is_time_7:
        is_daytime = True
        print(f"[TIME UPDATE] Auto-setting is_daytime=True because is_time_7=True (daytime starts at 7:00)")
    state["is_time_7"] = is_time_7
    state["is_daytime"] = is_daytime
    retract_fact(FACT_IS_TIME_7)
    retract_fact(FACT_IS_DAYTIME)
    if is_daytime:
        tell_fact(FACT_IS_DAYTIME)
        print(f"[TIME UPDATE] Daytime detected")
    if is_time_7:
        tell_fact(FACT_IS_TIME_7)
        print(f"[TIME UPDATE] Time 7:00 detected (wake-up time)")

def update_environmental_facts(weather, user_at_home, is_time_7=False, is_daytime=True):
    print(f"[ENV UPDATE] Setting weather='{weather}', user_at_home={user_at_home}, is_time_7={is_time_7}, is_daytime={is_daytime}")
    state["weather"] = weather
    retract_fact(FACT_FREEZING_WEATHER)
    retract_fact(FACT_COLD_WEATHER)
    retract_fact(FACT_WARM_WEATHER)
    retract_fact(FACT_HOT_WEATHER)
    if weather == WEATHER_FREEZING:
        tell_fact(FACT_FREEZING_WEATHER)
    elif weather == WEATHER_COLD:
        tell_fact(FACT_COLD_WEATHER)
    elif weather == WEATHER_WARM:
        tell_fact(FACT_WARM_WEATHER)
    elif weather == WEATHER_HOT:
        tell_fact(FACT_HOT_WEATHER)
    state["user_at_home"] = user_at_home
    retract_fact(FACT_USER_AT_HOME)
    retract_fact(FACT_USER_NOT_HOME)
    if user_at_home:
        tell_fact(FACT_USER_AT_HOME)
    else:
        tell_fact(FACT_USER_NOT_HOME)
    update_time_facts(is_time_7, is_daytime)

def update_user_location(user_at_home):
    print(f"[USER LOCATION] Updating user location: user_at_home={user_at_home}")
    state["user_at_home"] = user_at_home
    retract_fact(FACT_USER_AT_HOME)
    retract_fact(FACT_USER_NOT_HOME)
    if user_at_home:
        tell_fact(FACT_USER_AT_HOME)
    else:
        tell_fact(FACT_USER_NOT_HOME)

In [ ]:
# Uncomment these lines if you want to use Gemini API and have the necessary setup
# from google import genai
# from pydantic import BaseModel


def get_available_facts():
    available_facts = [
        FACT_FREEZING_WEATHER,
        FACT_COLD_WEATHER,
        FACT_WARM_WEATHER,
        FACT_HOT_WEATHER,
        FACT_USER_AT_HOME,
        FACT_USER_NOT_HOME,
        FACT_USER_LEAVING,
        FACT_USER_ARRIVING,
        FACT_IS_TIME_7,
        FACT_IS_DAYTIME,
        FACT_USER_WANTS_LAMP_ON,
        FACT_USER_WANTS_LAMP_OFF,
        FACT_USER_WANTS_LAMP_DIMMED,
        FACT_USER_WANTS_HEATING,
        FACT_USER_WANTS_COOLING,
        FACT_USER_WANTS_VENTILATION,
        FACT_USER_WANTS_THERMOSTAT_OFF,
        FACT_USER_WANTS_SECURITY_ARMED,
        FACT_USER_WANTS_SECURITY_HOME_MODE,
        FACT_USER_WANTS_SECURITY_DISARMED,
        FACT_USER_WANTS_TV_ON,
        FACT_USER_WANTS_TV_OFF,
        FACT_USER_WANTS_CURTAIN_OPEN,
        FACT_USER_WANTS_CURTAIN_CLOSED,
    ]
    print(
        f"[AVAILABLE FACTS] System understands {len(available_facts)} different facts"
    )
    return available_facts


# Gemini integration function (stub, replace with real API call if needed)
def generete_facts_from_gemini(user_input):
    print(f"[GEMINI] Generating facts from user input: '{user_input}'")

    # need to pass available facts and user input to gemini, so gemini can return inferred facts
    available_facts = get_available_facts()

    class Facts(BaseModel):
        facts: list[str]

    # Prepare the prompt for Gemini to say infer facts from the user input. The infered facts should be from the available facts.
    prompt = f"""
    You are a smart home agent that infers logical facts from user input.
    User input: "{user_input}"
    Available facts: {available_facts}
    Infer the most relevant facts from the user input and return them as a JSON array of strings. Return empty array if no facts can be inferred.
    Example output: ["user_wants_heating", "cold_weather"]
    """

    print(f"[GEMINI] Prompt for Gemini: {prompt}")

    ai_api_key = "put-your-api-key-here"
    client = genai.Client(api_key=ai_api_key)
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        config={
            "response_mime_type": "application/json",
            "response_schema": list[Facts],
        },
    )
    # Use the response as a JSON string.
    print(response.text)

    # Use instantiated objects.
    infered_facts_by_llm = response.parsed[0].facts
    print(f"[GEMINI] Inferred facts: {infered_facts_by_llm}")
    return infered_facts_by_llm


def infer_facts_from_input(user_input):
    user_input = user_input.lower().strip()
    genereted_facts = generete_facts_from_gemini(user_input)
    return genereted_facts

In [9]:
def process_user_input(user_input):
    print(f"\n[USER INPUT] Processing: '{user_input}'")
    inferred_facts = infer_facts_from_input(user_input)
    if not inferred_facts:
        print(f"[USER INPUT] No facts inferred from input: '{user_input}'")
        return
    for fact in inferred_facts:
        tell_fact(fact)
    if FACT_USER_LEAVING in inferred_facts:
        update_user_location(False)
    elif FACT_USER_ARRIVING in inferred_facts:
        update_user_location(True)
    evaluate_rules()
    print_state("AFTER USER INPUT")

def setup_system_rules():
    print(f"[SETUP] Setting up system rules...")
    tell_rule("freezing_weather AND user_at_home -> turn_on_heater")
    tell_rule("hot_weather AND user_at_home -> turn_on_cooler")
    tell_rule("user_not_home -> curtain_closed")
    tell_rule("user_not_home -> turn_off_thermostat")
    tell_rule("is_daytime AND user_at_home -> turn_off_lamp")
    tell_rule("is_time_7 AND user_at_home AND freezing_weather -> turn_on_heater")
    tell_rule("is_time_7 AND user_at_home AND cold_weather -> turn_on_heater")
    tell_rule("is_time_7 AND user_at_home AND hot_weather -> turn_on_cooler")
    tell_rule("is_time_7 AND user_at_home AND warm_weather -> turn_on_ventilation")
    tell_rule("is_time_7 AND user_at_home -> curtain_open")
    tell_rule("user_wants_lamp_on -> turn_on_lamp")
    tell_rule("user_wants_lamp_off -> turn_off_lamp")
    tell_rule("user_wants_lamp_dimmed -> dim_lamp")
    tell_rule("user_wants_heating -> turn_on_heater")
    tell_rule("user_wants_cooling -> turn_on_cooler")
    tell_rule("user_wants_ventilation -> turn_on_ventilation")
    tell_rule("user_wants_thermostat_off -> turn_off_thermostat")
    tell_rule("user_wants_security_armed -> arm_security")
    tell_rule("user_wants_security_home_mode -> home_mode_security")
    tell_rule("user_wants_security_disarmed -> disarm_security")
    tell_rule("user_wants_tv_on -> turn_on_tv")
    tell_rule("user_wants_tv_off -> turn_off_tv")
    tell_rule("user_wants_curtain_open -> curtain_open")
    tell_rule("user_wants_curtain_closed -> curtain_closed")

def clear_knowledge_base():
    print(f"[CLEAR] Clearing knowledge base...")
    facts.clear()
    rules.clear()

In [10]:
def initialize_scenario(weather=WEATHER_WARM, user_at_home=True, is_time_7=False, is_daytime=True):
    print(f"\n[SCENARIO INIT] Initializing scenario: weather='{weather}', user_at_home={user_at_home}, is_time_7={is_time_7}, is_daytime={is_daytime}")
    clear_knowledge_base()
    setup_system_rules()
    reset_devices_to_default()
    update_environmental_facts(weather, user_at_home, is_time_7, is_daytime)
    print_state("INITIAL STATE")
    evaluate_rules()
    print_state("AFTER INITIALIZATION")

In [11]:
def scenario_1():
    print("\n" + "=" * 50)
    print("Scenario 1: Weather is freezing and user at home")
    print("=" * 50)
    initialize_scenario(WEATHER_FREEZING, True)

def scenario_2():
    print("\n" + "=" * 50)
    print("Scenario 2: User leaves home when weather is cold")
    print("=" * 50)
    initialize_scenario(WEATHER_COLD, True)
    process_user_input("i am leaving")

def scenario_3():
    print("\n" + "=" * 50)
    print("Scenario 3: User opens the curtain")
    print("=" * 50)
    initialize_scenario(WEATHER_WARM, True)
    process_user_input("open the curtain")

def scenario_4():
    print("\n" + "=" * 50)
    print("Scenario 4: User manually controls thermostat in hot weather")
    print("=" * 50)
    initialize_scenario(WEATHER_HOT, True)
    process_user_input("turn on ventilation")

def scenario_5():
    print("\n" + "=" * 50)
    print("Scenario 5: Wake-up routine at 7:00 with cold weather")
    print("=" * 50)
    initialize_scenario(WEATHER_COLD, True, is_time_7=True, is_daytime=True)



In [12]:
def run_all_scenarios():
    scenario_1()
    print()
    scenario_2()
    print()
    scenario_3()
    print()
    scenario_4()
    print()
    scenario_5()


run_all_scenarios()


Scenario 1: Weather is freezing and user at home

[SCENARIO INIT] Initializing scenario: weather='freezing', user_at_home=True, is_time_7=False, is_daytime=True
[CLEAR] Clearing knowledge base...
[SETUP] Setting up system rules...
[RULE ADDED] freezing_weather AND user_at_home -> turn_on_heater
[RULE ADDED] hot_weather AND user_at_home -> turn_on_cooler
[RULE ADDED] user_not_home -> curtain_closed
[RULE ADDED] user_not_home -> turn_off_thermostat
[RULE ADDED] is_daytime AND user_at_home -> turn_off_lamp
[RULE ADDED] is_time_7 AND user_at_home AND freezing_weather -> turn_on_heater
[RULE ADDED] is_time_7 AND user_at_home AND cold_weather -> turn_on_heater
[RULE ADDED] is_time_7 AND user_at_home AND hot_weather -> turn_on_cooler
[RULE ADDED] is_time_7 AND user_at_home AND warm_weather -> turn_on_ventilation
[RULE ADDED] is_time_7 AND user_at_home -> curtain_open
[RULE ADDED] user_wants_lamp_on -> turn_on_lamp
[RULE ADDED] user_wants_lamp_off -> turn_off_lamp
[RULE ADDED] user_wants_lamp